In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import random
import pandas as pd
import numpy as np
import dill
import tensorflow as tf
from tensorflow import keras

### This notebook is only dedicated to submitting. The model training is made in : [JMarket: RNN with keras](https://www.kaggle.com/tchaye59/jmarket-rnn-with-keras)

# Load model and other files

In [ ]:
feature_cols = dill.load(open('/kaggle/input/jmarket-rnn-with-keras/feature_cols.dill','rb'))
model = tf.keras.models.load_model('/kaggle/input/jmarket-rnn-with-keras/train_model.hdf5')

# Generate submission

In [ ]:
n_steps = 10

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set
X = None
for (test_df, sample_prediction_df) in iter_test:
    
    if test_df.iloc[0].weight != 0:
        test_df.fillna(0,inplace=True)
        data = test_df[feature_cols].values
        if X is None:
            X = np.concatenate([data for _ in range(n_steps)],axis=0)
        X = np.concatenate([X[1:],data] ,axis=0)
    
        action = model(X[np.newaxis,:],training=False).numpy()[0,0] > 0.5
        action = int(action)
    else:
        action = 0
    
    sample_prediction_df.action = action
    env.predict(sample_prediction_df)

In [ ]:
#pd.read_csv('./submission.csv')